# i. Perkenalan

Nama : Akbar Fitriawan  
Batch : HCK-15

Tentang dataset:


Objective:



---

# ii. Import Libraries

In [ ]:
# library dataframe
import pandas as pd
# library olah data numerical
import numpy as np
# library statistik
from scipy.stats import pearsonr, kendalltau, spearmanr

# Data visualisasi
import matplotlib.pyplot as plt
import seaborn as sns

# libraries model mechine learning
from sklearn.linear_model import LinearRegression

# libraries Data Preprocessing
from sklearn.model_selection import train_test_split
from feature_engine.outliers import Winsorizer # outlier
from sklearn.preprocessing import StandardScaler, MinMaxScaler,RobustScaler # scalling
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder # encoder

# Libraries model evaluation
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error, mean_squared_error

# Display QQ plot
import statsmodels.api as sm

# Libraries model saving
import pickle
import json


# Library to suppress warnings or deprecation notes
import warnings
warnings.filterwarnings('ignore')



---

# iii. Data Loading

In [ ]:
df = pd.read_csv('')
df

In [ ]:
print('Jumlah Baris: ',df.shape[0])
print('Jumlah Kolom: ',df.shape[1])

In [ ]:
# Data 5 teratas 
df.head()

In [ ]:
# Data 5 Terbawah
df.tail()

In [ ]:
# Data info
df.info()

In [ ]:
df.describe()

---

# iv. Exploratory Data Analysis (EDA)

In [ ]:
# import pandas as pd

# # Membuat DataFrame contoh
# data = {'nama': ['Andi', 'Budi', 'Caca', 'Deni'],
#         'usia': [10, 20, 30, 60]}
# df = pd.DataFrame(data)

# # Fungsi untuk mengkategorikan usia
# def kategorikan_usia(usia):
#     if usia <= 12:
#         return 'anak-anak'
#     elif usia <= 20:
#         return 'remaja'
#     elif usia <= 50:
#         return 'dewasa'
#     else:
#         return 'orang tua'

# # Membuat kolom baru dengan hasil filter
# df['kategori_usia'] = df['usia'].apply(kategorikan_usia)

# print(df)


In [ ]:
# import pandas as pd

# # Membuat DataFrame contoh
# data = {'nama': ['Andi', 'Budi', 'Caca', 'Deni'],
#         'berat_badan_kg': [70, 80, 90, 100],
#         'tinggi_badan_m': [1.65, 1.75, 1.70, 1.80]}
# df = pd.DataFrame(data)

# # Fungsi untuk menghitung BMI
# def hitung_bmi(berat_badan, tinggi_badan):
#     return berat_badan / (tinggi_badan ** 2)

# # Fungsi untuk mengklasifikasikan overweight
# def klasifikasikan_overweight(bmi):
#     if 30 <= bmi < 35:
#         return 'overweight class 1'
#     elif 35 <= bmi < 40:
#         return 'overweight class 2'
#     elif bmi >= 40:
#         return 'overweight class 3'
#     else:
#         return 'normal'

# # Membuat kolom baru dengan kategori overweight
# df['bmi'] = df.apply(lambda row: hitung_bmi(row['berat_badan_kg'], row['tinggi_badan_m']), axis=1)
# df['overweight_category'] = df['bmi'].apply(klasifikasikan_overweight)

# print(df)


---

# v. Feature Engineering

In [ ]:
# cek duplicated data
df.duplicated().sum()

## Spliting Data

In [ ]:
# dump dataset agar tidak mengulang
data_copy = df.copy()

In [ ]:
# split data feature dan target
X = df.drop('y', axis=1)
y = df['y']

In [ ]:
# Split data Train dan Test
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
# Create functiion sizes chek
def print_data_sizes(X_train, y_train, X_test, y_test):
    print('Train Data')
    print('Ukuran X_train : ', X_train.shape)
    print('Ukuran y_train : ', y_train.shape)
    print('\n')
    print('Test Data')
    print('Ukuran X_test : ', X_test.shape)
    print('Ukuran y_test : ', y_test.shape)


print_data_sizes(X_train, y_train, X_test, y_test)

## Checking cardinality

In [ ]:
df.nunique()

## Handling Missing Value

In [ ]:
# cek missing values 
X_train.isnull().sum()

In [ ]:
# cek missing values 
X_test.isnull().sum()

In [ ]:
# cek missing values 
y_train.isnull().sum()

In [ ]:
# cek missing values 
y_train.isnull().sum()

## Handilng Outlier

In [ ]:
# Filter Numerical Columns into Variable
num_cols = X_train.select_dtypes(include=np.number).columns.to_list()

# Making data and columns for normal distribution
dataNum_normal = []
listCols_normal = []

# Making data and columns for skewed distribution
dataNum_skewed = []
listCols_skewed = []

# For loop in every numerical column to filer the data distribution into either normal distributed or skewed columns
for cols in num_cols:
    skewness = X_train[cols].skew()
    
    # If the data normally distributed
    if skewness <= 0.5 and skewness >= -0.5:            
        kolom_numerik_normal.append(cols)
        dataNum_normal.append([cols, skewness])
        
    # Elif the data is skewed
    elif skewness < -0.5 or skewness > 0.5:
        listCols_skewed.append(cols)
        dataNum_skewed.append([cols, skewness])

In [ ]:
# Showing Normally distributed columns
data_norm = pd.DataFrame(data=dataNum_normal, columns=['kolom_terdistribusi_normal', 'skewness'])
data_norm

In [ ]:
# Showing skewed columns
data_skew = pd.DataFrame(data=dataNum_skewed, columns=['kolom_skewed', 'skewness'])
data_skew

In [ ]:
# Input low variation columns into variable
kolom_low_variation = ['surge_multiplier', 'precipIntensity', 'precipProbability', 'uvIndex']

# Drop the low variation columns in skewed columns, and add it into normal distribution columns
for kolom in kolom_low_variation:
    listCols_skewed.remove(kolom) #-> removing low variation columns in skewed columns
    listCols_normal.append(kolom) #-> appending low variation columns in normal distributed columns

In [ ]:
# Capping Method for Normal Distribution  
winsorizer_normal_dist = Winsorizer(capping_method='gaussian',
                            tail='both',
                            fold=3,
                            variables=listCols_normal,
                            missing_values='ignore')

# Fit & Transforming X_train 
X_train_capped = winsorizer_normal_dist.fit_transform(X_train_cleaned)

# Transforming X_test
X_test_capped = winsorizer_normal_dist.transform(X_test_cleaned)

In [ ]:
winsorizer_skewed = Winsorizer(capping_method='iqr',
                            tail='both',
                            fold=1.5,
                            variables=listCols_skewed,
                            missing_values='ignore')

# Fit & Transforming X_train 
X_train_capped = winsorizer_skewed.fit_transform(X_train_cleaned)

# Transforming X_test
X_test_capped = winsorizer_skewed.transform(X_test_cleaned)

In [ ]:
# Plot Distribution Comparison
def outlier_handling_plot_comparison(df_before, df_after, variable):
    """
    This function is created to plot histograms and boxplots for a variable before and after outlier handling
    
    Parameters:
    - df_before (pandas.DataFrame): DataFrame before outlier handling
    - df_after (pandas.DataFrame): DataFrame after outlier handling
    - variable (str): The variable to plot
    
    Example: 
    num_cols = ['numeric_column']                                   <- Enter the numeric column
    for col in num_cols:                                            <- make a for loop in numeric column to access every columns
        plot_distribution_comparison(X_train, X_train_capped, col)  <- put X_train, X_train_capped, and columns in the variable

    """
    # Figure Size, and Super Title based on variable
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))                               
    fig.suptitle(f'{variable} - Distribution Before and After Outlier Handling')

    # Plot Histogram Before
    sns.histplot(df_before[variable], bins=30, ax=axes[0, 0], color='skyblue')
    axes[0, 0].set_title('Histogram Before')

    # Plot Boxplot Before
    sns.boxplot(y=df_before[variable], ax=axes[1, 0], color='lightgreen')
    axes[1, 0].set_title('Boxplot Before')

    # Plot Histogram After
    sns.histplot(df_after[variable], bins=30, ax=axes[0, 1], color='skyblue')
    axes[0, 1].set_title('Histogram After')

    # Plot Boxplot After
    sns.boxplot(y=df_after[variable], ax=axes[1, 1], color='lightgreen')
    axes[1, 1].set_title('Boxplot After')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
# Using the Function
num_cols = kolom_numerik 
for col in num_cols:
    outlier_handling_plot_comparison(X_train_cleaned, X_train_capped, col)

## Split Data Numerikal dan Kategorikal

In [ ]:
# column numeric list
num_columns = X_train_capped.select_dtypes(include=np.number).columns.tolist()
# column categorical list
cat_columns = X_train_capped.select_dtypes(include=['object']).columns.tolist()

print('Numerical List: ', num_columns)
print('Catgorical List: ', cat_columns)

In [ ]:
# override X_train with spliting list
X_train_num = X_train_capped[num_columns]
X_train_cat = X_train_capped[cat_columns]
# override X_test with spliting list
X_test_num = X_test_capped[num_columns]
X_test_cat = X_test_capped[cat_columns]

In [ ]:
# Checking sizes
print_data_sizes(X_train, y_train, X_test, y_test)

## Feature Selection


### Categories Feature

In [ ]:
# Finding the correlation between categorical columns and 'price' using Kendall Tau's correlation

p_values = []
interpretation = []
cols = []
corr = []


for col in X_train_cat.columns:
  corr_coef, p_value = kendalltau(X_train_cat[col], y_train)

  p_values.append(p_value)
  cols.append(col)
  corr.append(corr_coef)

  if p_value < 0.05:
    interpretation.append('Significant Correlation')
  else :
    interpretation.append('No Significant Correlation')

cat_corr= pd.DataFrame({'Column Name':cols,
              'Correlation Coefficient' : corr,
              'P-value':p_values,
              'Interpretation': interpretation })

cat_corr

In [ ]:
# filter corelation categorical
cat_corr[cat_corr['Interpretation'] == "Significant Correlation"]

### Numerical Feature

In [ ]:
# Finding the correlation between numerical columns and 'price' using pearsonr and spearmanr correlation

p_values = []
interpretation = []
cols = []
corr = []

for col in X_train_num.columns:
  if abs(X_train_num[col].skew()) < 0.5:                    #For Normally Distributed Columns
    corr_coef, p_value = pearsonr(X_train_num[col], y_train)

    p_values.append(p_value)
    cols.append(col)
    corr.append(corr_coef)

    if p_value < 0.05:
      interpretation.append('Significant Correlation')
    else :
      interpretation.append('No Significant Correlation')
  else:                                                     #For Non Normally Distributed Columns
    corr_coef, p_value = spearmanr(X_train_num[col], y_train)

    p_values.append(p_value)
    cols.append(col)
    corr.append(corr_coef)

    if p_value < 0.05:
      interpretation.append('Significant Correlation')
    else :
      interpretation.append('No Significant Correlation')

num_corr = pd.DataFrame({'Column Name':cols,
              'Correlation Coefficient' : corr,
              'P-value':p_values,
              'Interpretation': interpretation })

num_corr

In [ ]:
# filter corelation numerical
num_corr[num_corr['Interpretation'] == "Significant Correlation"]


#### Membuat variable baru Untuk Feature yang berkolerasi

In [ ]:
# Membuat list Feature berkorelasi dari numerikal dan kategorikal
selected_col_cat = ['cab_type','name','source','destination']
selected_col_num = ['distance','surge_multiplier']

In [ ]:
# append ke x_train dan 
X_train_cat = X_train_cat[selected_col_cat]
X_train_num = X_train_num[selected_col_num]

X_test_cat = X_test_cat[selected_col_cat]
X_test_num = X_test_num[selected_col_num]

In [ ]:
# Checking the categorical data
X_train_cat

In [ ]:
# Checking the numerical data
X_train_num

### Scaling dan Encoding

#### Scaling

In [ ]:
# Scaling menggunakan Standar Scaler karena lebih cocok untuk model linear regression 

#Initialize the Standar Scaler
scaler = StandardScaler()

#Fit_transform for X_train, transform for X_test
X_train_num_scaled = scaler.fit_transform(X_train_num) ## only fit in train
X_test_num_scaled = scaler.transform(X_test_num)

X_train_num_scaled

#### Encoding 

In [ ]:
# dump data agar tidak ngulang
dummp1 = X_train_cat.copy()
dummp2 = X_test_cat.copy()

In [ ]:
# Menggunakan One Hot Encoder karena fitur kategorikal kami tampaknya tidak memiliki hierarki yang ditentukan

#Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)


#Fit_transform for X_train, transform for X_test
X_train_cat_encoded = encoder.fit_transform(X_train_cat)
X_test_cat_encoded = encoder.transform(X_test_cat)


#Fit_transform for X_train, transform for X_test

X_train_cat_encoded

### Concatenate Categorical and Numerical

In [ ]:
X_train_final = np.concatenate([X_train_num_scaled,X_train_cat_encoded],axis=1)
X_test_final = np.concatenate([X_test_num_scaled, X_test_cat_encoded],axis=1)

# Transform into dataframe
X_train_final = pd.DataFrame(X_train_final)
X_test_final = pd.DataFrame(X_test_final)

In [ ]:
X_train_final

In [ ]:
X_test_final

---

# vi. Model Definition

In [ ]:
# inisialisasi model
model_linreg = LinearRegression()

---

# vii. Model Training

In [ ]:
# fitting model train
model_linreg.fit(X_train_final, y_train)

#### coefficients/slope dan intercept

In [ ]:
# Check coefficients and intercept

coefficients = model_linreg.coef_
intercept = model_linreg.intercept_

params = {
    'feature': X_train_final.columns.tolist(),
    'coefficient': model_linreg.coef_.tolist()
}

print('Intercept : ', intercept)
pd.DataFrame(params)

---

# viii. Model Evaluation

In [ ]:
# model evaluasi Train set
y_pred_train = model_linreg.predict(X_train_final)

# model evaluasi Test set
y_pred_test = model_linreg.predict(X_test_final)

In [ ]:
y_pred_train

In [ ]:
y_pred_test

In [ ]:
y_pred_test.shape

In [ ]:
y_pred_train.shape

In [ ]:
print('MAE - Train Set  : ', mean_absolute_error(y_train, y_pred_train))
print('MAE - Test Set   : ', mean_absolute_error(y_test, y_pred_test))
print('')

print('MSE - Train Set  : ', mean_squared_error(y_train, y_pred_train))
print('MSE - Test Set   : ', mean_squared_error(y_test, y_pred_test))
print('')

print('RMSE - Train Set : ', mean_squared_error(y_train, y_pred_train, squared=False))
print('RMSE - Test Set  : ', mean_squared_error(y_test, y_pred_test, squared=False))
print('')

print('R2 Score - Train Set : ', r2_score(y_train, y_pred_train))
print('R2 Score - Test Set  : ', r2_score(y_test, y_pred_test))
print('')

In [ ]:
# Plot evaluasi model dengan R2 Score
r2 = r2_score(y_test, y_pred_test)

# plot data
plt.figure()
plt.scatter(y_test, y_pred_test, c='blue', label='Data')
plt.plot(y_test, y_test, color='red', linewidth=2, label='Perfect Prediction')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title(f'R^2 Score: {r2:.2f}')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Evaluasi model dengan asumsi Multivariate Normality

# get residual
y_test_residuals = y_test - y_pred_test
y_test_residuals

In [ ]:
# Plot QQ plot residual

sm.qqplot(y_test_residuals, line ='s')
plt.show()

---

# ix. Model Saving

In [ ]:
#Model saving

with open('list_num_cols.txt', 'w') as file_1:
  json.dump(selected_col_num, file_1)

with open('list_cat_cols.txt', 'w') as file_2:
  json.dump(selected_col_cat, file_2)

with open('scaler.pkl', 'wb') as file_3:
  pickle.dump(scaler, file_3)

with open('encoder.pkl', 'wb') as file_4:
  pickle.dump(encoder, file_4)

with open('model.pkl', 'wb') as file_5:
  pickle.dump(model_linreg, file_5)

---

# x. Model Inference

In [ ]:

# Load model and other files

with open('list_cat_cols.txt', 'r') as file_1:
  list_cat_col = json.load(file_1)

with open('list_num_cols.txt', 'r') as file_2:
  list_num_col = json.load(file_2)

with open("model.pkl", "rb") as file_3:
  model = pickle.load(file_3)

with open("scaler.pkl", "rb") as file_4:
  scaler = pickle.load(file_4)

with open("encoder.pkl", "rb") as file_5:
  encoder = pickle.load(file_5)

In [ ]:
# # cara 1
# # Drop a random row
# random_index = np.random.randint(0, len(df))
# inference = df.iloc[random_index]

# # Drop the row from the DataFrame
# df.drop(index=random_index, inplace=True)

# # Save the modified DataFrame to CSV
# df.to_csv('modified_dataset.csv', index=False)

# # Save the dropped row (inference) to CSV
# inference.to_csv('inference.csv', index=False)

In [ ]:
# # cara 2

# # Making a randomized dataset for inference
# random_data = [
#     {
#         'PassengerId': i + 1,
#         'Survived': np.random.randint(0, 2),
#         'Pclass': np.random.randint(1, 4),
#         'Name': 'Name_' + str(i + 1),
#         'Sex': np.random.choice(['male', 'female']),
#         'Age': np.random.normal(loc=30, scale=10),
#         'SibSp': np.random.randint(0, 5),
#         'Parch': np.random.randint(0, 3),
#         'Ticket': 'Ticket_' + str(i + 1),
#         'Fare': np.random.uniform(0, 100),
#         'Cabin': np.random.choice(['A', 'B', 'C', 'D', 'E', 'F', 'G'], p=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.4]) + str(np.random.randint(1, 100)),
#         'Embarked': np.random.choice(['S', 'C', 'Q'])
#     }
#     for i in range(2)  # Adjust the range according to the desired number of rows
# ]

# # Convert to DataFrame
# random_df = pd.DataFrame(random_data)

# # Display the DataFrame
# random_df.head()

#### Splitting Dataset Into Categorical and Numerical Columns

In [ ]:
#Splitting the dataset into categorical and numerical columns

data_infNum = data_inf[list_num_col]
data_infCat = data_inf[list_cat_col]

#### Scaling and Encoding

In [ ]:
# Scaling the numerical features, encoding the categorical features, and concatenating them back again

data_inf_scaled_num = scaler.transform(data_infNum)
data_inf_cat_encoded = encoder.transform(data_infCat)
data_inference_final = np.concatenate([data_inf_scaled_num, data_inf_cat_encoded], axis = 1)

#### Predicting

In [ ]:
#Predicting

y_pred_inference = model.predict(data_inference_final)
y_pred_inference

---

# xi. Kesimpulan

### Conclusions